In [1]:
import requests
from bs4 import BeautifulSoup
import time
import html2text
import random
import ware_house
import re
from time import sleep, strftime
desktop_agents = ware_house.desktop_agents

In [2]:
def replace_other(text):
        """This function replace all not necessary symbols in the text"""
        for i in ware_house.replace_list:
            text = text.replace(i, "")

        return (text)


def random_headers():
    """This function create a random header for my parser"""
    return {'User-Agent': random.choice(desktop_agents),
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'}


def handle_text(text):
    """This function handle text and make it in the good format"""
    h = html2text.HTML2Text()
    h.ignore_links = True
    h.escape_all = True
    text = h.handle(text)
    text = replace_other(text)
    return (text)


def request_action(url, timeout):
    r = requests.get(url, headers=random_headers(), timeout=timeout)
    soup = BeautifulSoup(r.text, 'html.parser')
    return (soup)

In [3]:
# medicine = "Дюфастон"
# pages_amount = 3200

medicine = "Утрожестан"
pages_amount = 2500

# medicine = "Праджисан"
# pages_amount = 500

# medicine = "Ипрожин"
# pages_amount = 40

# Аккуратность вызывает безумие и стимулирует упорядочивание мыслей

In [4]:
import pandas as pd
id_posts = []
time_posts = []
name_author_list = []
post_previews_list = []
href_post_list = []
title_short_list = []
type_posts_lists = []
comments_number_post_list = []
pages = []
count = 0

dictio = {'ID_posts': id_posts,
                  'Author_name': name_author_list,
                  'Title_post' : title_short_list,
                  'Amount_of_comments':comments_number_post_list,
                  'Post_Preview': post_previews_list,
                  'Time_of_post': time_posts,
                  'Type_of_post': type_posts_lists,
                  'Href_post': href_post_list
                 }
df1 = pd.DataFrame.from_dict(dictio)


import pandas as pd

for i in range (0, pages_amount):
        try:
            main_web_page =  "https://www.babyblog.ru"
            page_number = i
            web_page = main_web_page + "/search/all/"+ str(page_number) + "?query=" + medicine + "&sort=date"
            url = web_page  
            soup = request_action(url, 30)

        #         print (soup.prettify())

            spans = soup.find_all('span', attrs={'class': "blog-created rel fl"}) 



            for span in spans:
                text = str(span)[str(span).find("""fl""")+4:str(span).find("""</span>""")]
                time_posts.append (text)

            spans = soup.find_all('div', attrs={'class': "content x64 fr"})  


            for i in list (spans [0].children)[1].findAll("div"):

                if 'data-object_type' in (i.attrs):
                    id_posts.append (str(str(i.attrs['data-object_type']) + '|' + str(i.attrs['data-object_id']) + '|' + str(i.attrs['data-object_user_id']) + '|' + str(i.attrs['data-shard_object_id']) + '|' + str(i.attrs['data-owner_user_id'])))


            spans = soup.find_all('div', attrs={'class': "content x64 fr"}) 


            pages.append (i)

            for i in list (spans [0].children)[1].findAll("div"):

                if 'class' in (i.attrs):
                    if 'bb-questions-item' in i.attrs['class']:
                        post_previews_list.append ("None")
                        type_posts_lists.append ('question')

                        name_author_list.append (list(i.children)[5].get_text())
                        comments_number_post_list.append(0)


                if 'class' in (i.attrs):
                    if 'user-used' in i.attrs['class']:

                        try:
                            comments = str(list(i.children)[5].get_text())
                            comments_number_post_list.append(int(comments[comments.find("ть ")+3:comments.find(" ком")]))

                        except:
                            comments_number_post_list.append(0)

                        post_previews_list.append ((list(i.children)[1].get_text()))

                        type_posts_lists.append ('post')


                if 'class' in (i.attrs):
                    if 'created' in i.attrs['class']:

                        name_author_list.append (list(i.findAll('a', attrs={'title': ""}))[1].get_text())

                if 'data-title' in (i.attrs):
                    title_short_list.append (i.attrs['data-title'])


                if 'data-url' in (i.attrs):
                    href_post_list.append (i.attrs['data-url'])



            if len(id_posts)-len(time_posts) != 0 or (len(href_post_list) - len(post_previews_list)) != 0 or (len(name_author_list)-len(title_short_list)) != 0:
                    print ('Error -- ',page_number)
                    id_posts = []
                    time_posts = []
                    name_author_list = []
                    post_previews_list = []
                    href_post_list = []
                    title_short_list = []
                    type_posts_lists = []
                    comments_number_post_list = []

            else:
                dictio_1 = {'ID_posts': id_posts,
                      'Author_name': name_author_list,
                      'Title_post' : title_short_list,
                      'Amount_of_comments':comments_number_post_list,
                      'Post_Preview': post_previews_list,
                      'Time_of_post': time_posts,
                      'Type_of_post': type_posts_lists,
                      'Href_post': href_post_list
                     }

                df2 = pd.DataFrame.from_dict(dictio_1)


                df1 = pd.concat([df1, df2])

                id_posts = []
                time_posts = []
                name_author_list = []
                post_previews_list = []
                href_post_list = []
                title_short_list = []
                type_posts_lists = []
                comments_number_post_list = []



        except:
            print ('It doesnt work, page:', count)
            count += 1
            pass
print (len(id_posts))
print (len(time_posts))
print (len(name_author_list))
print (len(post_previews_list) )
print (len(href_post_list))
print (len(title_short_list))
print (len(type_posts_lists))
print (len(comments_number_post_list))

Error --  430
Error --  501
It doesnt work, page: 0
It doesnt work, page: 1
It doesnt work, page: 2
Error --  673
Error --  715
It doesnt work, page: 3
Error --  873
Error --  879
Error --  1015
It doesnt work, page: 4
Error --  1065
Error --  1068
It doesnt work, page: 5
Error --  1110
It doesnt work, page: 6
Error --  1119
Error --  1149
Error --  1164
It doesnt work, page: 7
It doesnt work, page: 8
It doesnt work, page: 9
It doesnt work, page: 10
It doesnt work, page: 11
It doesnt work, page: 12
It doesnt work, page: 13
It doesnt work, page: 14
It doesnt work, page: 15
It doesnt work, page: 16
It doesnt work, page: 17
It doesnt work, page: 18
It doesnt work, page: 19
It doesnt work, page: 20
It doesnt work, page: 21
It doesnt work, page: 22
It doesnt work, page: 23
It doesnt work, page: 24
It doesnt work, page: 25
It doesnt work, page: 26
It doesnt work, page: 27
It doesnt work, page: 28
It doesnt work, page: 29
It doesnt work, page: 30
It doesnt work, page: 31
It doesnt work, page:

In [7]:
# print (df1.tail())
# df1.to_excel('SQL\\Excel_backups\\'+medicine+ '_{}.xlsx'.format(strftime("%Y%m%d-%H%M%S")))

df1['medicine'] = medicine

df1.to_csv('SQL\\Excel_backups\\'+ medicine + '_{}.csv'.format('hand'), sep=';', encoding='utf-8')

# sd
# updated_user_df.to_csv('{}_users_followed_list.csv'.format(strftime("%Y%m%d-%H%M%S")))

In [8]:
df1

,ID_posts,Author_name,Title_post,Amount_of_comments,Post_Preview,Time_of_post,Type_of_post,Href_post,medicine
0,2|3215781|2542713|4|2542713,ChuDo,14дпп и мы в стационаре,1.0,"Вчера мазало целый день, в ход пошёл весь арсе...",14 минут назад,post,/community/post/sterility/3215781?srcid=114183,Утрожестан
1,2|3215773|2437481|4|2437481,Ната,"Кому нужен вессел дуэ ф; клексан утрожестан ,...",0.0,\n,45 минут назад,post,/community/post/sterility/3215773?srcid=114183,Утрожестан
2,1|60441|2437481|2437481|2437481,Ната,"Отдам утрожестан, клексан , вессел дуэф",0.0,\n,49 минут назад,post,/user/myfamily888/60441?srcid=114183,Утрожестан
3,2|3215763|2697871|4|2697871,Юлия 2909,"Впервые такое, что за ХГЧ, неужели чуда не слу...",5.0,Моя непонятная динамика роста ХГЧ:\nПеренос 26...,1 час назад,post,/community/post/sterility/3215763?srcid=114183,Утрожестан
4,2|3413089|1251157|3489|1251157,Liana,Обмен лекарствами Москва,0.0,"Девочки, может кому-то нужно. Осталось:\n- Мет...",1 час назад,post,/community/post/happypregn/3413089?srcid=114183,Утрожестан
5,2|3215759|3072929|4|3072929,Nafanya,Вопрос про прогестерон. Или как дурная голова ...,0.0,"\n\nДевочки, хотела у вас уточнить по поводу п...",2 часа назад,post,/community/post/sterility/3215759?srcid=114183,Утрожестан
6,2|3215745|718899|4|718899,Екатерина,Чем поддержать малыша?,25.0,"Здравствуйте, девочки! Ситуация такая... Крио ...",4 часа назад,post,/community/post/sterility/3215745?srcid=114183,Утрожестан
7,2|3215739|1572379|4|1572379,Оксана Кузнецова Науменко,Анализы на 4 ДПП,8.0,"Девочки, привет.\nОбращаюсь к вам за помощью в...",7 часов назад,post,/community/post/sterility/3215739?srcid=114183,Утрожестан
8,2|3215733|2934675|4|2934675,Аня,прогестерон аллергия,4.0,"Девочки, скажите бывает такое - у меня 8нед.2д...",10 часов назад,post,/community/post/sterility/3215733?srcid=114183,Утрожестан
9,2|3195411|2012369|4|2012369,Танюшка,"меняю на памперсы, Украина",7.0,"УТРОЖЕСТАН 100 и 200, прогинова, ГОНАЛ Ф 75 ,г...",14 часов назад,post,/community/post/sterility/3195411?srcid=114183,Утрожестан


## SQL PART

In [11]:
import pyodbc

driver = 'SQL Server'
server = 'abtru1a11'
db1 = 'PEKHTDX'
tcon = 'yes'
uname = 'ONEABBOTT\PEKHTDX'
pword = ''
connStr = pyodbc.connect(driver='{SQL Server}', host=server, database=db1, trusted_connection=tcon, user=uname,
                      password=pword) 


cursor = connStr.cursor()
query = """DELETE FROM Page_BB_update"""
cursor.execute(query)
cursor.close()

print ("Delete all rows")

cursor = connStr.cursor()
for index,row in df1.iterrows():
    cursor.execute("""INSERT INTO dbo.Page_BB_update
    (
       Page_BB_update.[ID_posts]
      ,Page_BB_update.[Author_name]
      ,Page_BB_update.[Title_post]
      ,Page_BB_update.[Amount_of_comments]
      ,Page_BB_update.[Post_Preview]
      ,Page_BB_update.[Time_of_post]
      ,Page_BB_update.[Type_of_post]
      ,Page_BB_update.[Href_post]
      --,Page_BB_update.[medicine]
    ) 
                         values (?,?, ?,?, ?,?,?,?)""", row['ID_posts'], 
                                           row['Author_name'], 
                                           row['Title_post'], 
                                           row['Amount_of_comments'],
                                           row['Post_Preview'], 
                                           row['Time_of_post'], 
                                           row['Type_of_post'], 
                                           row['Href_post']
                                           #,row['medicine']
                  ) 
    connStr.commit()
cursor.close()



print ("Transfer data to the transfer table")


# cursor = connStr.cursor()
# query = """INSERT INTO Page_BB (Page_BB.[ID_posts]
#       ,Page_BB.[Author_name]
#       ,Page_BB.[Title_post]
#       ,Page_BB.[Post_Preview]
#       ,Page_BB.[Time_of_post]
#       ,Page_BB.[Type_of_post]
#       ,Page_BB.[Href_post]
#       --,Page_BB.[medicine]
#       )

# SELECT Page_BB_update.[ID_posts]
#       ,Page_BB_update.[Author_name]
#       ,Page_BB_update.[Title_post]
#       ,Page_BB_update.[Post_Preview]
#       ,Page_BB_update.[Time_of_post]
#       ,Page_BB_update.[Type_of_post]
#       ,Page_BB_update.[Href_post]
#       --,Page_BB_update.[medicine]
#   FROM Page_BB_update
#  WHERE Page_BB_update.[ID_posts] not in (  
#   SELECT distinct Page_BB.[ID_posts]
#    FROM Page_BB)"""

# cursor.execute(query)
# cursor.close()
# connStr.close()

# print ("Paste new to data")



Delete all rows


Error: ('01000', '[01000] [Microsoft][ODBC SQL Server Driver][DBNETLIB]ConnectionRead (recv()). (10053) (SQLEndTran); [01000] [Microsoft][ODBC SQL Server Driver][DBNETLIB]General network error. Check your network documentation. (11)')

In [ ]:
medicine

# Resources:
http://www.compjour.org/warmups/govt-text-releases/intro-to-bs4-lxml-parsing-wh-press-briefings/
/n

https://www.crummy.com/software/BeautifulSoup/bs4/doc/